In [7]:
import_libraries_for_ifc = True

if import_libraries_for_ifc:

        import os
        import sys
        import re

        import pandas as pd
        import numpy as np

        from pathlib import Path
        import json

        import ifcopenshell
        from shapely import wkt
        from shapely.geometry import MultiPolygon, Polygon
        from shapely.geometry.base import BaseGeometry
        from shapely.wkt import loads as load_wkt


In [8]:
name = 'first_floor_v13'  
current_directory = Path.cwd()

csv_folder = current_directory / 'outputs' / 'jsons'
csv_file = csv_folder / f'{name}.csv'
print(csv_file)

df = pd.read_csv(csv_file)
df_transformed = df.copy()
print('working file: {csv_file}')


c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\outputs\jsons\first_floor_v13.csv


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\oscar\\OneDrive - Fondazione Bruno Kessler\\KUL_GeometricModel\\outputs\\jsons\\first_floor_v13.csv'

In [ ]:
extract_all_elemnts_from_df = True

if extract_all_elemnts_from_df:

    df = df_transformed.copy()

    def safe_load_wkt(val):
        try:
            if isinstance(val, BaseGeometry):
                return val
            if isinstance(val, str) and val.strip():
                geom = load_wkt(val)
                return MultiPolygon([geom]) if isinstance(geom, Polygon) else geom
        except Exception as e:
            return None
        return None

    df["geometry"] = df["geometry_wkt"].apply(safe_load_wkt)

    def classify_surface(row):
        if row["thematic_surface"] == "closure" and isinstance(row["parent_id"], str) and row["parent_id"].startswith("room_"):
            return "room_closure"
        return row["thematic_surface"]

    df["thematic_surface"] = df.apply(classify_surface, axis=1)

    def refine_wall_labels(row):
        label = row["thematic_surface"]
        has_holes = bool(row.get("has_holes"))

        if label in ["party_wall", "party_wall_with_holes"]:
            return "external_wall" if has_holes else "internal_wall"
        elif label in ["external", "external_with_holes"]:
            return "external_wall" if has_holes else "external_wall"  # both map to external_wall

        return label

    df_element_ids = df[["element_id", "element_gml_id"]].drop_duplicates()
    df["thematic_surface"] = df.apply(refine_wall_labels, axis=1)
    df = df[df["geometry"].notna()]

    wall_types = ["external_wall", "internal_wall"]
    df["wall_type_detailed"] = df.apply(
        lambda row: f"{row['thematic_surface']}_with_holes"
        if bool(row.get("has_holes")) and row["thematic_surface"] in wall_types
        else row["thematic_surface"],
        axis=1
    )

    df_walls = df[df["wall_type_detailed"].isin([
        "external_wall", "external_with_holes",
        "internal_wall", "party_wall_with_holes"
    ])]

    thematic_walls = []

    for _, row in df_walls.iterrows():
        for poly in row["geometry"].geoms:
            coords = list(poly.exterior.coords)
            thematic_walls.append({
                "element_id": row["element_id"],
                "parent_id": row["parent_id"],
                "wall_type": row["wall_type_detailed"],
                "has_holes": row["has_holes"],
                "surface_vertices": coords
            })

    df_thematic_walls = pd.DataFrame(thematic_walls)

    print("✅ Wall types found:")
    print(df_thematic_walls["wall_type"].value_counts())

    # Define additional surface types to extract
    surface_types_to_extract = [
        "external_wall", "internal_wall", "external_with_holes", "party_wall", "party_wall_with_holes", "closure",
        "ceiling", "floor", "roof", 'door', 'window'  # add any additional thematic surfaces as needed
    ]

    # Filter the main dataframe to include the target surfaces
    df_surfaces = df[df["thematic_surface"].isin(surface_types_to_extract)]

    # Container for all extracted surfaces
    thematic_surfaces = []

    for _, row in df_surfaces.iterrows():
        for poly in row["geometry"].geoms:
            coords = list(poly.exterior.coords)
            thematic_surfaces.append({
                "element_id": row["element_id"],
                "parent_id": row["parent_id"],
                "element_gml_id": row['element_gml_id'],
                "thematic_surface": row["thematic_surface"],
                "element_type": row.get("wall_type_detailed", None),  # wall type if available
                "has_holes": row.get("has_holes", False),
                "surface_vertices": coords
            })

    # Create DataFrame with all extracted thematic surfaces
    df_thematic_surfaces = pd.DataFrame(thematic_surfaces)

    # Optional: print counts
    print("✅ Extracted surface types:")
    print(df_thematic_surfaces["thematic_surface"].value_counts())

df_thematic_surfaces

In [ ]:
internal_walls = df_thematic_surfaces[df_thematic_surfaces['thematic_surface'] == 'internal_wall']
external_walls = df_thematic_surfaces[df_thematic_surfaces['thematic_surface'] == 'external_wall']
floors = df_thematic_surfaces[df_thematic_surfaces['thematic_surface'] == 'floor']
ceilings = df_thematic_surfaces[df_thematic_surfaces['thematic_surface'] == 'ceiling']
windows = df_thematic_surfaces[df_thematic_surfaces['thematic_surface'] == 'window']
doors = df_thematic_surfaces[df_thematic_surfaces['thematic_surface'] == 'door']

df_doors = library_df[library_df['name'].str.contains('door', case=False, na=False)]

internal_thicknesses_round = internal_walls['thickness_rounded']
external_thicknesses_round = external_walls['thickness_rounded']

walls = pd.concat([internal_walls, external_walls], ignore_index=True)

In [ ]:

# --- IFC Setup ---
name = 'first_floor'
ifc_file = ifcopenshell.file(schema='IFC4')

project = ifc_file.create_entity("IfcProject", Name=name)
context = ifc_file.create_entity(
    "IfcGeometricRepresentationContext",
    ContextIdentifier='Body',
    ContextType='Model',
    CoordinateSpaceDimension=3,
    Precision=1e-5,
    WorldCoordinateSystem=ifc_file.create_entity(
        "IfcAxis2Placement3D",
        Location=ifc_file.create_entity("IfcCartesianPoint", Coordinates=[0.0, 0.0, 0.0])
    )
)

# --- Units ---
length_unit = ifc_file.create_entity("IfcSIUnit", UnitType="LENGTHUNIT", Name="METRE")
angle_unit = ifc_file.create_entity("IfcSIUnit", UnitType="PLANEANGLEUNIT", Name="RADIAN")
ifc_file.create_entity("IfcUnitAssignment", Units=[length_unit, angle_unit])

# --- Spatial hierarchy ---
building = ifc_file.create_entity("IfcBuilding", Name=name)
storey = ifc_file.create_entity("IfcBuildingStorey", Name=name)

all_elements = []

# --- Wall Creator ---
def create_wall_from_multipolygon(wkt_geom, name="Wall", height=3.0, thickness=0.2):
    geom = wkt.loads(wkt_geom)
    if not isinstance(geom, (Polygon, MultiPolygon)):
        raise ValueError(f"Unsupported geometry type for wall '{name}'")

    polygon = geom.geoms[0] if isinstance(geom, MultiPolygon) else geom
    coords = list(polygon.exterior.coords)

    max_len = 0
    best_segment = None
    for i in range(len(coords) - 1):
        p1, p2 = np.array(coords[i][:2]), np.array(coords[i + 1][:2])
        seg_len = np.linalg.norm(p2 - p1)
        if seg_len > max_len:
            max_len = seg_len
            best_segment = (p1, p2)

    if not best_segment:
        raise ValueError(f"No valid segment found for wall '{name}'")

    p1, p2 = best_segment
    direction = (p2 - p1) / np.linalg.norm(p2 - p1)
    midpoint = (p1 + p2) / 2
    length = np.linalg.norm(p2 - p1)

    placement = ifc_file.create_entity(
        "IfcLocalPlacement",
        PlacementRelTo=None,
        RelativePlacement=ifc_file.create_entity(
            "IfcAxis2Placement3D",
            Location=ifc_file.create_entity("IfcCartesianPoint", Coordinates=list(map(float, [midpoint[0], midpoint[1], 0.0]))),
            RefDirection=ifc_file.create_entity("IfcDirection", DirectionRatios=list(map(float, [direction[0], direction[1], 0.0])))
        )
    )

    wall = ifc_file.create_entity("IfcWall", Name=name, ObjectPlacement=placement)

    profile = ifc_file.create_entity(
        "IfcRectangleProfileDef",
        ProfileType="AREA",
        XDim=float(length),
        YDim=float(thickness)
    )

    solid = ifc_file.create_entity(
        "IfcExtrudedAreaSolid",
        SweptArea=profile,
        Position=ifc_file.create_entity(
            "IfcAxis2Placement3D",
            Location=ifc_file.create_entity("IfcCartesianPoint", Coordinates=[0.0, 0.0, 0.0])
        ),
        ExtrudedDirection=ifc_file.create_entity("IfcDirection", DirectionRatios=[0.0, 0.0, 1.0]),
        Depth=float(height)
    )

    shape = ifc_file.create_entity(
        "IfcShapeRepresentation",
        ContextOfItems=context,
        RepresentationIdentifier="Body",
        RepresentationType="SweptSolid",
        Items=[solid]
    )

    wall.Representation = ifc_file.create_entity(
        "IfcProductDefinitionShape",
        Representations=[shape]
    )

    return wall

# --- Filter walls ---
wall_types = ['external', 'party_wall', 'external_with_hole', 'party_wall_with_hole', 'closure']
df_wall = df_building_with_thickness[df_building_with_thickness["thematic_surface"].isin(wall_types)]

for i, row in df_wall.iterrows():
    raw_name = row.get("name")
    name = str(raw_name) if pd.notna(raw_name) else f"Unnamed Wall[{i + 1}]"
    thickness = row.get("thickness_m", 0.2)
    thickness = thickness if pd.notna(thickness) else 0.2
    wkt_geom = row.get("geometry_wkt")

    if isinstance(wkt_geom, str) and "MULTIPOLYGON" in wkt_geom:
        try:
            wall_entity = create_wall_from_multipolygon(wkt_geom, name=name, thickness=thickness)
            all_elements.append(wall_entity)
        except Exception as e:
            print(f"❌ Skipping wall '{name}': {e}")
    else:
        print(f"⚠️ Invalid or missing WKT for wall '{name}'")

# --- Add ceiling, floor, doors, windows, rooms ---
for df, ifc_type, label in [
    (df_floor, "IfcSlab", "Unnamed Floor"),
    (df_ceiling, "IfcCovering", "Unnamed Ceiling"),
    (df_door, "IfcDoor", "Unnamed Door"),
    (df_window, "IfcWindow", "Unnamed Window"),
    (df_room, "IfcSpace", "Unnamed Room")
]:
    for _, row in df.iterrows():
        name = str(row.get("name")) if pd.notna(row.get("name")) else label
        entity = ifc_file.create_entity(ifc_type, Name=name)
        all_elements.append(entity)

# --- Spatial containment ---
ifc_file.create_entity(
    "IfcRelContainedInSpatialStructure",
    GlobalId=ifcopenshell.guid.new(),
    RelatingStructure=storey,
    RelatedElements=all_elements
)

ifc_file.create_entity(
    "IfcRelAggregates",
    GlobalId=ifcopenshell.guid.new(),
    RelatingObject=building,
    RelatedObjects=[storey]
)

ifc_file.create_entity(
    "IfcRelAggregates",
    GlobalId=ifcopenshell.guid.new(),
    RelatingObject=project,
    RelatedObjects=[building]
)

# --- Write output ---
ifc_file.write(f"{name}.ifc")
print(f"✅ IFC file written: {name}.ifc")
